In [5]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
train_y_df = pd.read_csv(r"/kaggle/input/dog-breed-identification/labels.csv")

train_y_df['filename'] = train_y_df['id'] + '.jpg'

num_classes = train_y_df['breed'].nunique()
train_dir = "/kaggle/input/dog-breed-identification/train"

# Image and Batch Size
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
OUTPUT_PATH = '/kaggle/working/'
EPOCHS = 30

#Rescaling and data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 80/20 split
    horizontal_flip=True,
    zoom_range=0.2,
    rotation_range=15
)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_y_df,
    directory=train_dir,
    x_col='filename',
    y_col='breed',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

val_generator = datagen.flow_from_dataframe(
    dataframe=train_y_df,
    directory=train_dir,
    x_col='filename',
    y_col='breed',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)
# EfficientNetB0 base model with pretrained ImageNet weights
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3))

#Unfreezing for training
base_model.trainable = True

#custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
output = Dense(num_classes, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=output)


model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy',tf.keras.metrics.TopKCategoricalAccuracy(k=5)])
checkpoint = ModelCheckpoint(
    OUTPUT_PATH + 'best_model.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Training
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    # callbacks=[checkpoint, early_stop]
)
#Saving Training
with open(OUTPUT_PATH + 'history.pkl', 'wb') as f:
    pickle.dump(history.history, f)


plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title("Model Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.grid(True)
plt.show()

#Saving Training
with open(OUTPUT_PATH + 'history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

Found 8178 validated image filenames belonging to 120 classes.
Found 2044 validated image filenames belonging to 120 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30


I0000 00:00:1751859719.479977      99 service.cc:148] XLA service 0x7ad4880026b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751859719.480682      99 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1751859719.480710      99 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1751859724.572943      99 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1751859734.700606      99 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1751859734.844233      99 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1751859735.303060      99 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. Th

224/256 ━━━━━━━━━━━━━━━━━━━━ 15s 483ms/step - accuracy: 0.0080 - loss: 4.8519 - top_k_categorical_accuracy: 0.0439

E0000 00:00:1751859877.293478      99 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1751859877.433033      99 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1751859877.778127      99 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1751859877.918913      99 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


256/256 ━━━━━━━━━━━━━━━━━━━━ 293s 779ms/step - accuracy: 0.0081 - loss: 4.8501 - top_k_categorical_accuracy: 0.0442 - val_accuracy: 0.0068 - val_loss: 4.7954 - val_top_k_categorical_accuracy: 0.0386
Epoch 2/30
256/256 ━━━━━━━━━━━━━━━━━━━━ 122s 478ms/step - accuracy: 0.0139 - loss: 4.7870 - top_k_categorical_accuracy: 0.0631 - val_accuracy: 0.0083 - val_loss: 4.8017 - val_top_k_categorical_accuracy: 0.0396
Epoch 3/30
256/256 ━━━━━━━━━━━━━━━━━━━━ 122s 476ms/step - accuracy: 0.0238 - loss: 4.7329 - top_k_categorical_accuracy: 0.0834 - val_accuracy: 0.0181 - val_loss: 4.7325 - val_top_k_categorical_accuracy: 0.0890
Epoch 4/30
256/256 ━━━━━━━━━━━━━━━━━━━━ 138s 538ms/step - accuracy: 0.0282 - loss: 4.6900 - top_k_categorical_accuracy: 0.1065 - val_accuracy: 0.0660 - val_loss: 4.5835 - val_top_k_categorical_accuracy: 0.2021
Epoch 5/30
256/256 ━━━━━━━━━━━━━━━━━━━━ 137s 534ms/step - accuracy: 0.0509 - loss: 4.6020 - top_k_categorical_accuracy: 0.1600 - val_accuracy: 0.0533 - val_loss: 4.6008 - 

NameError: name 'pickle' is not defined

In [8]:
model.save('/kaggle/working/final_model.h5')

In [9]:
import os
print(os.listdir('/kaggle/working/'))

['.virtual_documents', 'history.pkl', 'final_model.h5']


In [10]:
import pickle

with open('/kaggle/working/history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

In [11]:
import os
print(os.listdir('/kaggle/working/'))

['.virtual_documents', 'history.pkl', 'final_model.h5']


In [26]:
predicted_labels = [i for i in predicted_indices]
print(predicted_labels)
submission = pd.DataFrame({
    'id': test_df['filename'].str.replace('.jpg', '', regex=False),
    'breed': predicted_labels
})

# Save as test_submission.csv
submission.to_csv('/kaggle/working/test_submission.csv', index=False)

[104, 89, 30, 60, 115, 90, 95, 78, 104, 94, 61, 117, 71, 16, 112, 104, 58, 8, 59, 95, 86, 16, 67, 69, 78, 116, 95, 59, 88, 0, 32, 97, 51, 86, 95, 47, 101, 82, 74, 44, 108, 0, 74, 30, 9, 86, 66, 105, 0, 7, 36, 90, 88, 117, 41, 44, 96, 55, 109, 102, 30, 33, 86, 24, 83, 52, 91, 45, 10, 36, 57, 57, 1, 17, 42, 48, 22, 31, 33, 15, 65, 52, 33, 100, 55, 73, 101, 93, 15, 91, 104, 55, 114, 107, 96, 79, 69, 69, 88, 2, 96, 95, 82, 10, 74, 96, 55, 66, 94, 3, 22, 39, 6, 14, 2, 92, 60, 98, 48, 1, 118, 94, 112, 45, 10, 94, 86, 16, 71, 80, 63, 44, 87, 27, 63, 88, 3, 12, 101, 55, 75, 69, 79, 38, 111, 28, 6, 74, 30, 6, 14, 10, 19, 37, 88, 50, 104, 85, 100, 78, 34, 34, 60, 41, 69, 100, 67, 95, 97, 104, 101, 36, 9, 87, 61, 92, 75, 75, 87, 118, 6, 30, 29, 96, 84, 11, 39, 114, 72, 14, 41, 68, 97, 119, 67, 55, 87, 91, 96, 71, 88, 7, 112, 79, 1, 83, 83, 42, 56, 95, 9, 96, 108, 42, 95, 109, 3, 84, 25, 61, 51, 13, 112, 106, 96, 73, 6, 65, 118, 88, 23, 8, 87, 88, 24, 19, 10, 101, 65, 18, 100, 50, 55, 74, 59, 108,